In [25]:
def scrapDarazData():
    from requests import get
    from bs4 import BeautifulSoup
    import re
    import json 
    from time import sleep,time
    from IPython.core.display import clear_output
    import pandas as pd

    id=[]
    name=[]
    brand=[]
    image=[]
    price=[]
    originalPrice=[]
    url=[]

    base_url="https://www.daraz.com.np/smartphones/?page="
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'} 

    pages=13
    start_time = time()
    requests = 0

    for page in range(1,pages+1):
        sleep(0.1)
        response = get(base_url+str(page),headers=headers)  
        page_html = BeautifulSoup(response.text, 'html.parser')

        script=page_html.findAll('script')[2].text
        text=re.search('"listItems":\[{.*}\],"re', script).group()
        text=text.strip('"listItems": ,"re')
        text+='}]'
        data=json.loads(text)

        for d in data:
            id.append(d['nid'])
            name.append(d['name'])
            brand.append(d['brandName'])
            image.append(d['image'])
            price.append(d['price'])        
            try:
                orp=d['originalPrice']
            except Exception:
                originalPrice.append(d['price'])
            else:
                originalPrice.append(orp) 

            url.append(d['productUrl'].replace('//','https://'))   
    
    darazData = pd.DataFrame({'id': id, 
                          'name': name, 
                          'brand':brand,
                          'image':image, 
                          'price':price, 
                          'originalPrice':originalPrice,
                          'url':url,
                          'web_id':1}) 
    return darazData
    
def scrapNepBayData():
    from requests import get
    from bs4 import BeautifulSoup
    from time import sleep
    import pandas as pd

    name=[]
    brand=[]
    url=[]
    price=[]
    originalPrice=[]

    pages=6
    base_url="https://nepbay.com/shopping/nepal/smart-phones-mobiles-in-nepal?page="

    for page in range(1,pages+1):   
        #make a get request    
        headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
        response = get(base_url+str(page),headers=headers)

        # Pause the loop to reduce server overhead
        sleep(0.1)

        # Parse the content of the request with BeautifulSoup
        page_html = BeautifulSoup(response.text, 'html.parser')
        data=page_html.findAll("div",{"class":"ncs-ad-list"}) 

        for d in data:
            n=d.find("div",{'class':'TGLBox'}).find('h4').text
            name.append(n)
            try:
                b=n.split(' ')[0].strip()         
            except Exception:
                brand.append(n)
            else:
                brand.append(b)
            pr=int(float(d.find("div",{'class':'TGLBox'}).find('p').find('span').text.replace(',','')))
            price.append(pr)
            originalPrice.append(pr)
            url.append(d.find("div",{'class':'TGLBox'}).find('h4').find('a')['href']) 
        
    nepbayData = pd.DataFrame({'name':name,
                      'brand':brand,
                      'price':price, 
                      'originalPrice':originalPrice,
                      'url':url,
                      'web_id':2})    
    return nepbayData
        
        
def scrapSastoDealData():
    import time
    from selenium import webdriver
    from selenium.webdriver.common.keys import Keys
    import pandas as pd

    brand=[]
    name=[]
    url=[]
    image=[]
    originalPrice=[]
    price=[]

    browser = webdriver.Chrome()
    browser.get("https://www.sastodeal.com/sastodeal/cta-mobiles-28?flag=setFilters&catalogueID=10&tct=4&apct=4&cts=28&emt=&searchkey=&defaultCategory=28&lvl=&bid=&size=&color=&sd=&bName=&spr=1000%20TO%20205000")
    elem = browser.find_element_by_tag_name("body")

    no_of_pagedowns = 90

    while no_of_pagedowns:
        elem.send_keys(Keys.PAGE_DOWN)
        time.sleep(0.1)
        no_of_pagedowns-=1

    products=browser.find_elements_by_class_name('categorytDetailDiv')

    for p in products:
        t=p.find_element_by_class_name("title").text
        name.append(t)
        url.append(p.find_element_by_class_name("fullWidth").get_attribute("href"))
        p1=p.find_element_by_class_name("offer-price").text.split()[1].replace(',','')
        price.append(p1)
        try:
            p2=p.find_element_by_class_name("linethrough").text.split()[1].replace(',','')
        except Exception:
            originalPrice.append(p1)
        else:
            originalPrice.append(p2)
        brand.append(t.split(" ")[0])

    sastoDealData = pd.DataFrame({'name': name, 
                        'url': url,  
                        'price':price, 
                        'originalPrice': originalPrice, 
                        'brand': brand,
                        'web_id':3})
    return sastoDealData

In [ ]:
from apscheduler.schedulers.blocking import BlockingScheduler
from fuzzywuzzy import fuzz
import re
import numpy as np
import pandas as pd

def namecleaner(name):
    name=re.sub(r'\(.*\)', '', name)
    name=re.sub(r'\[.*\]', '', name)
    return name

def scrapper():
    darazData=scrapDarazData()
    sleep(0.5)
    nepBayData=scrapNepBayData()
    sleep(0.5)
    sastoDealData=scrapSastoDealData()    
    
    maindf = pd.DataFrame(columns=['id','name','url','image','price','originalPrice','brand','web_id'])
    rows=[]
    for i in range(len(sdf)):
        ratio=[]
        for j in range(len(ddf)):
            ratio.append(fuzz.token_set_ratio(namecleaner(sdf['name'][i]),namecleaner(ddf['name'][j]))) 
        if(max(ratio)>=95):
            k=ratio.index(max(ratio))
            row=sdf.loc[i]
            row['id']=ddf['id'][k]
            rows.append(row)
            rows.append(ddf.loc[k])
            
    df1=pd.DataFrame(rows)
    df2=df1.copy()
    df1=df1.drop_duplicates(['name'], keep='first')
    df2=df2[df2.drop_duplicates().image.isnull()==False].reset_index(drop=True)
    
    import pymysql as sql
    connection = sql.connect(host='localhost',user='root',password='deven123',database='shoplab')
    
    #data insertion for product table
    for i in range(len(df1)):
        cursor = connection.cursor()
        id=np.str(df1['id'][i])
        name=np.str(df1['name'][i])
        brand=np.str(df1['brand'][i])
        description=np.str(df1['description'][i])
        image=np.str(df1['image'][i])
        sql=("INSERT INTO products(id,name,brand,description,image) VALUES(%s,%s,%s,%s,%s)")
        val=(id,name,brand,description,image)
        cursor.execute(sql,val)
        connection.commit()
    
    #data insertion for prices table
    for i in range(len(df2)):
        cursor = connection.cursor()
        productid=np.str(df2['id'][i])
        webid=np.str(df2['web_id'][i])
        price=np.str(df2['price'][i])
        originalprice=np.str(df2['originalPrice'][i])
        url=np.str(df2['url'][i])
        sql=("INSERT INTO prices(productid,webid,price,originalprice,url) VALUES(%s,%s,%s,%s,%s)")
        val = (productid, webid,price,originalprice,url)
        cursor.execute(sql, val)
        connection.commit()

scheduler = BlockingScheduler()
scheduler.add_job(scrapper, 'interval', hours=12)
scheduler.start()